In [3]:
import pandas as pd
file_path = '每日各站進出站人數/每日各站進出站人數2020.csv'
df1 = pd.read_csv(file_path)

In [4]:
df2 = df1.rename(columns={"trnOpDate":"乘車日",
            "staCode":"車站代碼",
            "gateInComingCnt":"進站人數",
            "gateOutGoingCnt":"出站人數"})
df2

,乘車日,車站代碼,進站人數,出站人數
0,20200101,900,7552,8154
1,20200101,910,1020,1135
2,20200101,920,1623,1972
3,20200101,930,4130,4813
4,20200101,940,1818,2128
...,...,...,...,...
86927,20201231,7360,3523,3602
86928,20201231,7361,3,18
86929,20201231,7362,28,34
86930,20201231,7380,609,604


In [5]:
import json 
from pprint import pprint

with open ('車站基本資料集.json',encoding='utf-8') as file:
    json_data = json.load(file)
stations_info = pd.DataFrame(json_data,columns=['stationCode','stationName'])
stations_info['stationCode'] = stations_info['stationCode'].map(lambda value:int(value))
stations_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stationCode  243 non-null    int64 
 1   stationName  243 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.9+ KB


In [6]:
import json
from pprint import pprint

with open('車站基本資料集.json',encoding='utf-8') as file:
   json_data = json.load(file)

stations_info = pd.DataFrame(json_data,columns=['stationCode','stationName'])
stations_info = stations_info.astype({'stationCode':'int32'})
stations_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stationCode  243 non-null    int32 
 1   stationName  243 non-null    object
dtypes: int32(1), object(1)
memory usage: 3.0+ KB


In [7]:
#使用pd.merge()
df1 = pd.merge(df2,stations_info,left_on='車站代碼',right_on='stationCode')
df2 = df1.rename(columns={'stationName':'車站名稱'})

In [8]:
#df2.drop(columns=['車站代碼','stationCode'])
df3=df2.reindex(columns=['乘車日','車站名稱','進站人數','出站人數'])
df3

,乘車日,車站名稱,進站人數,出站人數
0,20200101,基隆,7552,8154
1,20200101,三坑,1020,1135
2,20200101,八堵,1623,1972
3,20200101,七堵,4130,4813
4,20200101,百福,1818,2128
...,...,...,...,...
86927,20201231,瑞芳,3523,3602
86928,20201231,海科館,3,18
86929,20201231,八斗子,28,34
86930,20201231,四腳亭,609,604


In [9]:
df3['乘車日'] = pd.to_datetime(df3['乘車日'].astype(str))
df3

,乘車日,車站名稱,進站人數,出站人數
0,2020-01-01,基隆,7552,8154
1,2020-01-01,三坑,1020,1135
2,2020-01-01,八堵,1623,1972
3,2020-01-01,七堵,4130,4813
4,2020-01-01,百福,1818,2128
...,...,...,...,...
86927,2020-12-31,瑞芳,3523,3602
86928,2020-12-31,海科館,3,18
86929,2020-12-31,八斗子,28,34
86930,2020-12-31,四腳亭,609,604


In [3]:
import os
os.listdir('每日各站進出站人數')

['每日各站進出站人數2022.csv',
 '每日各站進出站人數2023.csv',
 '每日各站進出站人數20190423-20191231.csv',
 '每日各站進出站人數2021.csv',
 'manifest.csv',
 '每日各站進出站人數2020.csv',
 'schema.csv']

In [8]:
#產生2023年,以下金控公司,最高收盤價,最低收盤價,最高收盤價日期,最低收盤價日期
#提示,max(),min(),idxmin(),idxmax()

import yfinance as yf
import pandas as pd

def call(s):
    min=s.min()
    max=s.max()
    daymin=s.idxmin()
    daymax=s.idxmax()
    return pd.Series([max,min,daymin,daymax],index=['最高價','最低價','最低價日期','最高價日期'])

#國泰金(2992.TW),新光金(2888.TW),中信金(2891.TW),台新金(2887.TW)
all_data={ticker:yf.download(ticker) for ticker in ['2882.TW','2888.TW','2891.TW','2887.TW']}
all_data1 = {ticker:df['Adj Close'] for ticker,df in all_data.items()}
prices_df = pd.DataFrame(all_data1)
prices_df.columns = ['國泰金','新光金','中信金','台新金']
prices_df1 = prices_df.dropna()
prices_df1_2023 = prices_df1.loc['2023']
prices_df1_2023

#取得最高價,最低價 dataFrame
df_high_low_value=prices_df1_2023.apply(call)

#取得最低價日期 series, 再轉成[最低價日期] 
series_low_date=prices_df1_2023.idxmin()
df_low_date=pd.DataFrame([series_low_date], index=['最低價日期']).astype(str)

#取得最高價日期 series, 再轉成[最高價日期]
series_high_date=prices_df1_2023.idxmax()
df_high_date=pd.DataFrame([series_high_date], index=['最高價日期']).astype(str)

#結果
df_total=pd.concat([df_high_low_value,df_low_date,df_high_date], ignore_index=False)
df_total

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,國泰金,新光金,中信金,台新金
最高價,45.648304,9.83,27.176025,17.105072
最低價,37.808987,7.99,19.154709,13.100577
最低價日期,2023-01-03 00:00:00,2023-03-20 00:00:00,2023-03-17 00:00:00,2023-01-03 00:00:00
最高價日期,2023-08-01 00:00:00,2023-08-01 00:00:00,2023-12-14 00:00:00,2023-07-26 00:00:00
最低價日期,2023-01-03,2023-03-20,2023-03-17,2023-01-03
最高價日期,2023-08-01,2023-08-01,2023-12-14,2023-07-26
